# 병렬 처리가 필요한 상황 .
- 원격 서버에서 데이터 수집하는 경우
- 수집한 데이터에 이미지분석, OCR 처럼 길고 복잡한 작업이면서 작업을 수행하면서 동시에 데이터를 가져와야하는 경우
- 매 쿼리에 대해 비용을 지불하는 대규모 웹 서비스에서 데이터 수집, 사용 계약 안에서 다중연결을 이용하는 경우 

# 파이썬 (멀티프로세싱, 멀티쓰레딩 둘 다 지원) 
- 프로세스는 독립적으로 메모리를 할당받는다. 
- 그래.. 멀티프로세싱보다 멀티쓰레딩이 쉬운데, 편리함엔 대가가 따른다. 
- 파이썬의 전역 인터프리터 잠금(GIL)은 스레드가 같은 코드 행을 동시에 실행하지 못하게 막는 방식으로 스레드들이 공유하는 메모리가 손상되지 않게 보호한다. 
- 그래서 이 잠금 기능 덕분에 멀티스레드 프로그램을 작성할 때 각 라인이 의도한대로 잘 동작할거라 확신 할 수 있지만 (병목현상을 만들 가능성도 있다.)


# _thread 모듈 사용한다. (thread 모듈은 파기됬음) 



In [1]:
import _thread
import time 

def print_time(threadName, delay, iterations): 
    start = int(time.time())
    for i in range(0,iterations):
        time.sleep(delay)
        seconds_elapsed = str(int(time.time()) - start)
        print(f"{threadName} {time.ctime(time.time())} {seconds_elapsed}")
try:
    _thread.start_new_thread(print_time,('Fizz',3,33))
    _thread.start_new_thread(print_time,('Buzz',5,20))
    _thread.start_new_thread((print_time,('Counter',1,100)))
except:
    print("Error: unable to start thread")
while 1:
    pass # 
    
print_time('Counter',1,100)


Error: unable to start thread
Fizz Mon Jul 29 22:18:12 2024 3
Buzz Mon Jul 29 22:18:14 2024 5
Fizz Mon Jul 29 22:18:15 2024 6
Fizz Mon Jul 29 22:18:19 2024 10
Buzz Mon Jul 29 22:18:19 2024 10
Fizz Mon Jul 29 22:18:22 2024 13
Buzz Mon Jul 29 22:18:24 2024 15
Fizz Mon Jul 29 22:18:25 2024 16
Fizz Mon Jul 29 22:18:28 2024 19
Buzz Mon Jul 29 22:18:30 2024 21
Fizz Mon Jul 29 22:18:31 2024 22
Fizz Mon Jul 29 22:18:34 2024 25
Buzz Mon Jul 29 22:18:35 2024 26
Fizz Mon Jul 29 22:18:37 2024 28
Buzz Mon Jul 29 22:18:40 2024 31
Fizz Mon Jul 29 22:18:40 2024 31
Fizz Mon Jul 29 22:18:43 2024 34
Buzz Mon Jul 29 22:18:45 2024 36
Fizz Mon Jul 29 22:18:46 2024 37
Fizz Mon Jul 29 22:18:49 2024 40
Buzz Mon Jul 29 22:18:50 2024 41
Fizz Mon Jul 29 22:18:52 2024 43
Buzz Mon Jul 29 22:18:55 2024 46
Fizz Mon Jul 29 22:18:55 2024 46
Fizz Mon Jul 29 22:18:58 2024 49
Buzz Mon Jul 29 22:19:00 2024 51
Fizz Mon Jul 29 22:19:01 2024 52
Fizz Mon Jul 29 22:19:04 2024 55
Buzz Mon Jul 29 22:19:05 2024 56
Fizz Mon Jul 29 

KeyboardInterrupt: 

# 보통 크롤링 한 URL 을 다시 크롤링하지 않기 위해 리스트에 저장했었는데,
만약 2개쓰레드로 하면퍼센트보장 할 수 없다. (경쟁상태) 이건 디버깅도 안되고 머리아픔

## 리스트와 큐 
리스트는 효율적으로 읽고, 마지막 항목을 추가하는것이 효율적이지만 임의의 위치에 항목을 추가하는것은 비효율적이다. 
- 특히 리스트 처음에 추가하는 것은 매우 비효율적이다. 
- 심지어 끝에 추가하는것 조차도 경쟁상태에 빠질 수 있게해서 안전하지 않다. 

## 그럼 변수에?? 
- 변수도 마찬가지로 중간에 (바꿔넣기 전에) 다른 스레드가 바꿔버릴 수도 있음. (안전X) 

## 큐
- 큐는 스레드 간에 안전하게 데이터를 전달하는 방법이다.
- (큐의 설계 목적은 정적 데이터를 저장하는 것이 아니라 멀티스레드에서도 안전한 방식으로 데이터를 전송하자는 것이다.)
- 중간에 큐가 있고 DB 앞에 쓰레드가(인터페이스)가 대기하면서 큐에 넘어오는 데이터를 받아서 DB에 넣는게효율적임. 



In [1]:
# 그러니 2개의 쓰레드는 크롤링하고 1개는 DB에 저장하되 큐를 사용하면 예외없이 안전하게 데이터를 전달할 수 있다.! 

# Threading 모듈 
_thread 모듈은 스레드를 미세 조정할 수 있는 저수준 모듈인데, 간편한 고수준 모듈은 제공 X 
- threading 모듈은 _thread 모듈을 기반으로 만들어졌다. 고수준 , 깨끗하게 사용 할 수 있는 상위 인터페이스
- 다른 스레드에서 사용할 수 없는 로컬 스레드 데이터를 쉽게 만들 수 있다. 여러 스레드가 서로 다른 사이트를 스크랩하면서 각각 방문한 페이지 리스트를 관리할 때 유용함.
- 

In [2]:
import threading
import time

def print_time(threadName, delay , iterations):
    start = int(time.time())
    for i in range(0, iterations):
        time.sleep(delay)
        seconds_elapsed = str(int(time.time()) - start)
        print('{} {}'.format(seconds_elapsed, threadName))

threading.Thread(target=print_time, args = ('Fizz', 3, 33)).start()
threading.Thread(target=print_time, args = ('Buzz', 5, 20)).start()
threading.Thread(target=print_time, args = ('Counter', 1, 100)).start()


In [3]:
# - 다른 스레드에서 사용할 수 없는 로컬 스레드 데이터를 쉽게 만들 수 있다. 여러 스레드가 서로 다른 사이트를 스크랩하면서 각각 방문한 페이지 리스트를 관리할 때 유용함.
# 이러면 스레드의 공유 객체 간에 발생하는 경쟁 상태 문제를 해결 할 수 있음 (객체를 공유할 필요가 없어지면, 로컬 스레드 메모리로 옮겨야한다. 큐로 가능)
def crawler(url):
    data = threading.local()
    data.visited = []
    # 크롤링 코드
    
threading.Thread(target=crawler, args=('http://www.google.com',)).start()

# is_alive() : 스레드가 실행중인지 확인
# join() : 스레드가 종료될 때까지 대기
# is_Done() : 스레드가 종료되었는지 확인

In [ ]:
threading.Thread(target=crawler)
t.start()

# 충돌나면 다시 시작하게 가능 
while True: 
    time.sleep(1)
    if not t.isAlive():
        t = threading.Thread(target=crawler)
        t.start()
